In [54]:
import numpy as np
import pandas as pd
import json
from typing import Dict
from itertools import groupby
from functools import reduce
from collections import Counter, defaultdict
import re

In [55]:
df = pd.read_json("export/export_0.json")

# Gold Per Stage

In [5]:
def gold_spent(player):
    gold_spent_per_round = {}
    for currRound, group in groupby(player["gold"]["by_round"], lambda x: x["current_round"]):
        l = list(group)
        goldSpent = max(0,int(l[0]["gold"]) - int(l[-1]["gold"]))
        gold_spent_per_round[currRound] = goldSpent
    gold_spent_by_stage = defaultdict(int)
    for x in list(gold_spent_per_round.keys()):
        m = re.match(r'(\d).+', x)
        gold_spent_by_stage[f"gold_spent_stage_{m.group(1)}"] += gold_spent_per_round[m.group(0)]

    return gold_spent_by_stage

In [6]:
gold_spent(df.player.iloc[0])

defaultdict(int,
            {'gold_spent_stage_1': 4,
             'gold_spent_stage_2': 20,
             'gold_spent_stage_3': 13,
             'gold_spent_stage_4': 97,
             'gold_spent_stage_5': 40,
             'gold_spent_stage_6': 30})

In [11]:
gold_dict = [gold_spent(df.player.iloc[i]) for i in range(df.shape[0])]

# Level Per Stage

In [18]:
def level_stage(player):
    tp_dict = {}
    
    level_agg = 0
    current_stage = 0
    count_stage = 0
    
    for key, group in groupby(player["xp"]["by_round"], lambda x: x["current_round"]): 
        round_split = key.split("-")
        stage = int(round_split[0])

        #start of loop
        if current_stage == 0:
            current_stage = stage
        #going from stage x to stage y
        elif current_stage != stage:
            level_avg = level_agg/count_stage
            tp_dict[f"level_at_stage_{current_stage}"] = level_avg 
            current_stage = stage
            level_agg = 0
            count_stage = 0

        #continue
        list_r = list(group)
        level_agg += list_r[0]['level']
        count_stage += 1

    #getting the last stage
    level_avg = level_agg/count_stage
    tp_dict[f"level_at_stage_{current_stage}"] = level_avg
    
    return tp_dict

In [21]:
level_stage(df.player.iloc[0])

{'level_at_stage_1': 2.0,
 'level_at_stage_2': 4.166666666666667,
 'level_at_stage_3': 5.333333333333333,
 'level_at_stage_4': 6.5,
 'level_at_stage_5': 7.333333333333333,
 'level_at_stage_6': 8.0}

In [20]:
level_dict = [level_stage(df.player.iloc[i]) for i in range(df.shape[0])]

# Active Traits

In [12]:
def encode_traits(trait_dict_column, active_trait_df, trait_index):
    encoding_list = []
    for trait_dict in trait_dict_column:
        trait_encoding = [ 0 for i in range(len(trait_index)) ]
        for key, value in trait_dict.items():
            if key[:5] == 'Set3_':
                trait = key[5:]
            else:
                trait = key
            min_active_dict = active_trait_df.loc[active_trait_df['trait'] == trait]['min_active'].values[0]
            has_trait_flag = 0
            for k, v in min_active_dict.items():
                if value >= v[0] and value < v[1]:
                    trait_tier = int(k)
                    has_trait_flag = 1
            if has_trait_flag == 0:
                trait_tier = 0
            trait_encoding[trait_index[trait]] = trait_tier
        encoding_list.append(trait_encoding)
    return encoding_list

def get_active_traits(player, char_trait, trait_tier):
    active_traits = {}
    last_rounds = get_last_rounds(player)
    for rounds in last_rounds:
        stage = rounds.split('-')[0]
        board = player['round_outcomes']['by_round'][rounds]['board']
        char_seen = set()
        trait_count = {}
        for char_dict in board:
            char = char_dict['character_id']
            if char not in char_seen:
                char_seen.add(char_dict['character_id'])
                trait_list = char_trait.loc[char_trait['name'] == char]['trait'].values[0].replace('\'', '').replace(' ', '').strip('][ ').split(',')
                for trait in trait_list:
                    if trait not in trait_count:
                        trait_count[trait] = 1
                    else:
                        trait_count[trait] += 1      
        active_traits[stage] = trait_count
        
    return active_traits
    
def get_last_rounds(player):
    round_list = list(player['round_outcomes']['by_round'].keys())
    last_round_stage = []
    last_r = 0
    last_round = ''
    for rounds in round_list:
        r = int(rounds.split('-')[1])
        if last_r > r:
            last_round_stage.append(last_round)
        last_r = r
        last_round = rounds
    last_round_stage.append(round_list[-1])
    return last_round_stage

In [13]:
with open('full_mapping.json') as json_data:
    fullmap = json.load(json_data)
    
char_trait = pd.DataFrame(fullmap['character_trait_json'].items()).astype(str)
char_trait.columns = ['name', 'trait']
print(char_trait)

trait_tier = pd.DataFrame(fullmap['trait_tier_mapping_json'].items())
print(trait_tier)

                name                                          trait
0        TFT3_Thresh                       ['ManaReaver', 'Chrono']
1          TFT3_Ekko                  ['Infiltrator', 'Cybernetic']
2   TFT3_AurelionSol                          ['Starship', 'Rebel']
3          TFT3_Lulu                        ['Mystic', 'Celestial']
4         TFT3_Janna                    ['Paragon', 'StarGuardian']
5        TFT3_Soraka                     ['Mystic', 'StarGuardian']
6         TFT3_Urgot                    ['Protector', 'Battlecast']
7     TFT3_Gangplank  ['Mercenary', 'Demolitionist', 'SpacePirate']
8        TFT3_Xerath                       ['Sorcerer', 'DarkStar']
9          TFT3_Fizz                   ['Infiltrator', 'MechPilot']
10       TFT3_Irelia    ['ManaReaver', 'Blademaster', 'Cybernetic']
11       TFT3_Ezreal                          ['Blaster', 'Chrono']
12        TFT3_Riven                      ['Blademaster', 'Chrono']
13        TFT3_Fiora                  ['Blademas

In [15]:
get_active_traits(df.player.iloc[0], char_trait, trait_tier)

{'1': {'Blademaster': 1, 'Cybernetic': 1, 'Protector': 1, 'DarkStar': 1},
 '2': {'Blademaster': 1,
  'Celestial': 2,
  'Mystic': 1,
  'DarkStar': 3,
  'Protector': 2,
  'Vanguard': 1},
 '3': {'Blademaster': 1,
  'Celestial': 2,
  'Sniper': 1,
  'Chrono': 2,
  'Vanguard': 2,
  'Protector': 2,
  'DarkStar': 2},
 '4': {'Sniper': 1,
  'Celestial': 4,
  'Blademaster': 2,
  'ManaReaver': 1,
  'Cybernetic': 1,
  'Protector': 4,
  'DarkStar': 1,
  'StarGuardian': 1},
 '5': {'Sniper': 1,
  'Celestial': 4,
  'Blademaster': 3,
  'ManaReaver': 1,
  'Cybernetic': 1,
  'Protector': 4,
  'Battlecast': 1,
  'DarkStar': 1,
  'Chrono': 1},
 '6': {'Sniper': 1,
  'Celestial': 4,
  'Blademaster': 3,
  'ManaReaver': 1,
  'Cybernetic': 1,
  'Protector': 4,
  'Battlecast': 1,
  'DarkStar': 1,
  'Chrono': 1}}

In [22]:
def active_trait_func(player):
    return get_active_traits(player, char_trait, trait_tier)

In [23]:
active_trait_dict = [active_trait_func(df.player.iloc[i]) for i in range(df.shape[0])]

# Characters and Items

In [83]:
def character_items(player):
    tp_dict={}
    
    round_outcome_dict = player['round_outcomes']
    by_round_dict = round_outcome_dict['by_round']
    
    current_stage = 0
    prev_stage = 1
    
    current_round = 'a'
    prev_round = 'a'
    
    for key in by_round_dict.keys():
        round_split = key.split("-")
        current_stage = int(round_split[0])

        current_round = key

        if(prev_round == 'a'):
            prev_round = key
            prev_stage = current_stage

        if (current_stage == prev_stage):
            prev_round = key
        #went from stage x to stage y
        elif (current_stage != prev_stage):
            #a dictionary containg board
            #get the board characters and items
            board_dict = by_round_dict[prev_round]['board']
            #each index has one character and item combo in a dictionary with character_id and items
            #the temp dictionary will be 1: [character_id, items]

            temp_list = []
            
            for char in board_dict:
                character_id_value = char['character_id']
                items_value = char['items']

                temp_dict = {}
                temp_dict['character_id'] = character_id_value
                temp_dict['items'] = items_value

                temp_list.append(temp_dict)

            tp_dict[prev_stage] = temp_list

            prev_round = key
            prev_stage = current_stage
            
    #last one
    board_dict = by_round_dict[prev_round]['board']
    
    temp_list = []

    for char in board_dict:
        character_id_value = char['character_id']
        items_value = char['items']

        temp_dict = {}
        temp_dict['character_id'] = character_id_value
        temp_dict['items'] = items_value

        temp_list.append(temp_dict)

    tp_dict[prev_stage] = temp_list
    
    
    return tp_dict


{1: [{'character_id': 'TFT3_Fiora', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': []}],
 2: [{'character_id': 'TFT3_Xayah', 'items': [19]},
  {'character_id': 'TFT3_Karma', 'items': []},
  {'character_id': 'TFT3_XinZhao', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': []},
  {'character_id': 'TFT3_Mordekaiser', 'items': [77]}],
 3: [{'character_id': 'TFT3_Xayah', 'items': [19]},
  {'character_id': 'TFT3_Caitlyn', 'items': []},
  {'character_id': 'TFT3_WuKong', 'items': [6]},
  {'character_id': 'TFT3_JarvanIV', 'items': []},
  {'character_id': 'TFT3_XinZhao', 'items': []},
  {'character_id': 'TFT3_Mordekaiser', 'items': [77]}],
 4: [{'character_id': 'TFT3_Ashe', 'items': []},
  {'character_id': 'TFT3_Xayah', 'items': [19, 23]},
  {'character_id': 'TFT3_Irelia', 'items': [15, 19, 44]},
  {'character_id': 'TFT3_XinZhao', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': [66, 77]},
  {'character_id': 'TFT3_Rakan', 'items': []},
  {'character_id': 'TFT3_Nee

In [84]:
char_item_dict = [character_items(df.player.iloc[i]) for i in range(df.shape[0])]

In [86]:
char_item_dict[2]

{1: [{'character_id': 'TFT3_Caitlyn', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': []}],
 2: [{'character_id': 'TFT3_Xayah', 'items': [6, 19]},
  {'character_id': 'TFT3_Karma', 'items': []},
  {'character_id': 'TFT3_XinZhao', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': [5]}],
 3: [{'character_id': 'TFT3_Rakan', 'items': [37]},
  {'character_id': 'TFT3_Xayah', 'items': [6, 19]},
  {'character_id': 'TFT3_Riven', 'items': []},
  {'character_id': 'TFT3_Shen', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': [5]},
  {'character_id': 'TFT3_Fiora', 'items': []}],
 4: [{'character_id': 'TFT3_Rakan', 'items': [37]},
  {'character_id': 'TFT3_Riven', 'items': []},
  {'character_id': 'TFT3_Xayah', 'items': [12, 19, 69]},
  {'character_id': 'TFT3_Shen', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': [55]},
  {'character_id': 'TFT3_MasterYi', 'items': []}],
 5: [{'character_id': 'TFT3_Rakan', 'items': [37]},
  {'character_id': 'TFT3_MasterYi', 'item